In [ ]:
!pip install torch torchaudio soundfile pandas numpy matplotlib scikit-learn speechbrain lightning-bolts
!pip uninstall -y typeguard
!pip install typeguard==2.13.3  

In [ ]:
!git clone https://github.com/zyzisyz/mfa_conformer.git

In [ ]:
import os

print("Patching MFA-Conformer code for PyTorch Lightning compatibility...")

main_py_path = "/kaggle/working/mfa_conformer/main.py"

# --- MAIN.PY PATCH ---
with open(main_py_path, "r") as f:
    main_content = f.read()

# Replace DDPPlugin
main_content = main_content.replace(
    "from pytorch_lightning.plugins import DDPPlugin",
    "from pytorch_lightning.strategies import DDPStrategy"
)
main_content = main_content.replace(
    "plugins=DDPPlugin(find_unused_parameters=False)",
    "strategy=DDPStrategy(find_unused_parameters=False)"
)

with open(main_py_path, "w") as f:
    f.write(main_content)

print("Patched main.py for PyTorch Lightning >=1.8 compatibility.")

# --- LOADER.PY PATCH ---
loader_py_path = "/kaggle/working/mfa_conformer/module/loader.py"
if os.path.exists(loader_py_path):
    with open(loader_py_path, "r") as f:
        loader_content = f.read()

    if (
        "UnlabeledImagenet" in loader_content
        and "from pl_bolts.datasets import UnlabeledImagenet" in loader_content
    ):
        lines = loader_content.splitlines()
        new_lines = []
        for line in lines:
            if "from pl_bolts.datasets import UnlabeledImagenet" in line:
                new_lines.append("# " + line + "  # Commented out for compatibility")
            else:
                new_lines.append(line)
        with open(loader_py_path, "w") as f:
            f.write('\n'.join(new_lines))
        print("Patched loader.py for pl_bolts compatibility.")

print("Code patching complete.")

In [ ]:
import os
import argparse
import tqdm
import sys

def check_file_exists(base_dir, speaker_id, file_id):
    """Check if the audio file exists"""
    full_path = os.path.join(base_dir, speaker_id, file_id)
    return os.path.isfile(full_path)

def filter_t_file(input_file, output_file, base_dir):
    """Filter the training file (t) to remove lines referring to non-existent files"""
    with open(input_file, 'r') as fin, open(output_file, 'w') as fout:
        lines = fin.readlines()
        print(f"Processing {len(lines)} lines from {input_file}...")
        
        kept_lines = 0
        for line in tqdm.tqdm(lines, leave=False):
            parts = line.strip().split()
            if len(parts) == 2:
                speaker_id, file_id = parts
                if check_file_exists(base_dir, speaker_id, file_id):
                    fout.write(line)
                    kept_lines += 1
        
        print(f"Kept {kept_lines}/{len(lines)} lines in {output_file}")

def filter_eh_file(input_file, output_file, base_dir):
    """Filter the validation files (e/h) to remove lines referring to non-existent files"""
    with open(input_file, 'r') as fin, open(output_file, 'w') as fout:
        lines = fin.readlines()
        print(f"Processing {len(lines)} lines from {input_file}...")
        
        kept_lines = 0
        for line in tqdm.tqdm(lines, leave=False):
            parts = line.strip().split()
            if len(parts) == 3:
                label, file1, file2 = parts
                
                # Split speaker_id and file_id
                speaker_id1, file_id1 = file1.split('/')
                speaker_id2, file_id2 = file2.split('/')
                
                if (check_file_exists(base_dir, speaker_id1, file_id1) and 
                    check_file_exists(base_dir, speaker_id2, file_id2)):
                    fout.write(line)
                    kept_lines += 1
        
        print(f"Kept {kept_lines}/{len(lines)} lines in {output_file}")

def main():
    argv = [arg for arg in sys.argv if not arg.startswith('-f')]
    
    parser = argparse.ArgumentParser(description="Filter lines referring to non-existent files")
    parser.add_argument('--input_dir', type=str, default='/kaggle/input/vietnam-celeb-dataset/full-dataset',
                        help='Directory containing Vietnam-celeb dataset')
    parser.add_argument('--output_dir', type=str, default='/kaggle/working',
                        help='Output directory for filtered files')
    
    # Parse only known args and ignore unknown ones
    args, unknown = parser.parse_known_args(argv)
    
    # Paths to files and data directory
    data_dir = os.path.join(args.input_dir, 'data')
    t_file = os.path.join(args.input_dir, 'vietnam-celeb-t.txt')
    e_file = os.path.join(args.input_dir, 'vietnam-celeb-e.txt')
    h_file = os.path.join(args.input_dir, 'vietnam-celeb-h.txt')
    
    # Output files
    t_output = os.path.join(args.output_dir, 'filtered-vietnam-celeb-t.txt')
    e_output = os.path.join(args.output_dir, 'filtered-vietnam-celeb-e.txt')
    h_output = os.path.join(args.output_dir, 'filtered-vietnam-celeb-h.txt')
    
    # Create output directory if it doesn't exist
    os.makedirs(args.output_dir, exist_ok=True)
    
    # Filter the files
    print("Filtering training file (t)...")
    filter_t_file(t_file, t_output, data_dir)
    
    print("\nFiltering validation file (e)...")
    filter_eh_file(e_file, e_output, data_dir)
    
    print("\nFiltering validation file (h)...")
    filter_eh_file(h_file, h_output, data_dir)
    
    print("\nAll files filtered successfully!")

if __name__ == "__main__":
    main()

In [ ]:
import os
import argparse
import pandas as pd
import numpy as np
import tqdm

def create_training_csv(input_file, output_file, data_dir):
    """
    Create training CSV file in MFA Conformer format:
    - speaker_name: String label for the speaker
    - utt_paths: Path to audio file
    - utt_spk_int_labels: Integer label for the speaker
    """
    # Read filtered training file
    with open(input_file, 'r') as f:
        lines = f.readlines()
    
    # Process the lines
    speaker_names = []
    utt_paths = []
    
    print(f"Processing {len(lines)} lines from {input_file}...")
    
    for line in tqdm.tqdm(lines, leave=False):
        parts = line.strip().split()
        if len(parts) == 2:
            speaker_id, file_id = parts
            full_path = os.path.join(data_dir, speaker_id, file_id)
            speaker_names.append(speaker_id)
            utt_paths.append(full_path)
    
    # Create speaker to integer mapping
    unique_speakers = sorted(list(set(speaker_names)))
    speaker_to_int = {speaker: idx for idx, speaker in enumerate(unique_speakers)}
    
    # Create integer labels
    utt_spk_int_labels = [speaker_to_int[speaker] for speaker in speaker_names]
    
    # Create DataFrame
    df = pd.DataFrame({
        'speaker_name': speaker_names,
        'utt_paths': utt_paths,
        'utt_spk_int_labels': utt_spk_int_labels
    })
    
    # Save to CSV
    df.to_csv(output_file, index=False)
    print(f"Created training dataset with {len(df)} entries from {len(unique_speakers)} unique speakers")
    print(f"Saved to {output_file}")

def create_validation_file(input_files, output_file, data_dir):
    """
    Create validation file in MFA Conformer format:
    Similar to VoxCeleb trial format: label enroll_path test_path
    Can accept a list of input files and concatenate all lines.
    """
    all_lines = []
    for input_file in input_files:
        with open(input_file, 'r') as f:
            lines = f.readlines()
            all_lines.extend(lines)
        print(f"Loaded {len(lines)} lines from {input_file}")
    
    print(f"Processing {len(all_lines)} total lines from {input_files}...")
    
    with open(output_file, 'w') as f:
        for line in tqdm.tqdm(all_lines, leave=False):
            parts = line.strip().split()
            if len(parts) == 3:
                label, file1, file2 = parts
                
                # Split speaker_id and file_id
                speaker_id1, file_id1 = file1.split('/')
                speaker_id2, file_id2 = file2.split('/')
                
                # Create full paths
                enroll_path = os.path.join(data_dir, speaker_id1, file_id1)
                test_path = os.path.join(data_dir, speaker_id2, file_id2)
                
                # Write in MFA Conformer validation format
                f.write(f"{label} {enroll_path} {test_path}\n")
    
    print(f"Created validation file with {len(all_lines)} trials")
    print(f"Saved to {output_file}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Create MFA Conformer format datasets")
    parser.add_argument('--input_dir', type=str, default='/kaggle/working',
                        help='Directory containing filtered files')
    parser.add_argument('--output_dir', type=str, default='/kaggle/working',
                        help='Output directory for MFA Conformer format files')
    parser.add_argument('--data_dir', type=str, default='/kaggle/input/vietnam-celeb-dataset/full-dataset/data',
                        help='Directory containing audio data')
    args = parser.parse_args([])
    
    # Input files
    t_input = os.path.join(args.input_dir, 'filtered-vietnam-celeb-t.txt')
    e_input = os.path.join(args.input_dir, 'filtered-vietnam-celeb-e.txt')
    h_input = os.path.join(args.input_dir, 'filtered-vietnam-celeb-h.txt')
    
    # Output files
    train_output = os.path.join(args.output_dir, 'train.csv')
    e_val_output = os.path.join(args.output_dir, 'validation_e.txt')
    h_val_output = os.path.join(args.output_dir, 'validation_h.txt')
    full_val_output = os.path.join(args.output_dir, 'validation.txt')
    
    # Create output directory if it doesn't exist
    os.makedirs(args.output_dir, exist_ok=True)
    
    # Create training dataset
    print("Creating training dataset...")
    create_training_csv(t_input, train_output, args.data_dir)
    
    print("\nCreating validation dataset from 'e' file...")
    create_validation_file([e_input], e_val_output, args.data_dir)
    
    print("\nCreating validation dataset from 'h' file...")
    create_validation_file([h_input], h_val_output, args.data_dir)
    
    print("\nCreating combined validation dataset from both 'e' and 'h' files...")
    create_validation_file([e_input, h_input], full_val_output, args.data_dir)
    
    print("\nAll datasets created successfully!")

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from scipy.io import wavfile
import pickle
import random
from tqdm.notebook import tqdm
import gc
import psutil

# Import augmentation từ mfa_conformer
import sys
sys.path.append('/kaggle/working/mfa_conformer')
from module.augment import WavAugment
from module.dataset import load_audio

class PreAugmentedDataset:
    """
    Pre-augment 60% of data và cache trong RAM
    """
    def __init__(self, train_csv_path, second=3, aug_ratio=0.6, cache_dir="/kaggle/working/cached_data"):
        self.second = second
        self.aug_ratio = aug_ratio
        self.cache_dir = cache_dir
        
        # Đọc original training data
        df = pd.read_csv(train_csv_path)
        self.original_labels = df["utt_spk_int_labels"].values
        self.original_paths = df["utt_paths"].values
        
        print(f"📊 Original dataset: {len(self.original_paths)} samples")
        print(f"🔧 Will augment {aug_ratio*100:.0f}% of data")
        
        # Tạo cache directory
        os.makedirs(cache_dir, exist_ok=True)
        
        # Initialize augmentation
        self.wav_aug = WavAugment()
        
        # Storage cho cached data
        self.cached_waveforms = []
        self.cached_labels = []
        self.is_augmented = []  # Track which samples are augmented
        
    def check_memory_usage(self):
        """Check current memory usage"""
        process = psutil.Process(os.getpid())
        memory_info = process.memory_info()
        memory_gb = memory_info.rss / (1024**3)
        print(f"💾 Current memory usage: {memory_gb:.2f} GB")
        return memory_gb
    
    def estimate_memory_needed(self, sample_size=100):
        """Estimate memory needed for full dataset"""
        print("🔍 Estimating memory requirements...")
        
        # Test với sample nhỏ
        sample_indices = random.sample(range(len(self.original_paths)), min(sample_size, len(self.original_paths)))
        total_size = 0
        
        for idx in sample_indices:
            waveform = load_audio(self.original_paths[idx], self.second)
            total_size += waveform.nbytes
        
        avg_size_per_sample = total_size / len(sample_indices)
        
        # Estimate cho full dataset (original + 60% augmented)
        total_samples = len(self.original_paths) + int(len(self.original_paths) * self.aug_ratio)
        estimated_memory_gb = (total_samples * avg_size_per_sample) / (1024**3)
        
        print(f"📏 Average waveform size: {avg_size_per_sample/1024:.1f} KB")
        print(f"📊 Total samples (original + augmented): {total_samples}")
        print(f"💾 Estimated memory needed: {estimated_memory_gb:.2f} GB")
        
        return estimated_memory_gb
    
    def create_cached_dataset(self):
        """Pre-augment và cache toàn bộ dataset"""
        print("\n" + "="*60)
        print("🚀 CREATING PRE-AUGMENTED CACHED DATASET")
        print("="*60)
        
        # Check memory
        initial_memory = self.check_memory_usage()
        estimated_memory = self.estimate_memory_needed()
        
        if estimated_memory > 12:  # Kaggle has ~13GB RAM
            print(f"⚠️  Warning: Estimated memory ({estimated_memory:.1f}GB) might exceed Kaggle limits!")
            print("💡 Consider reducing aug_ratio or dataset size")
            
        print(f"\n📥 Loading and caching data...")
        
        # 1. Load tất cả original data
        print("1️⃣ Loading original data...")
        for i, (path, label) in enumerate(tqdm(zip(self.original_paths, self.original_labels), 
                                              total=len(self.original_paths), 
                                              desc="Loading original")):
            try:
                waveform = load_audio(path, self.second)
                self.cached_waveforms.append(waveform.copy())
                self.cached_labels.append(label)
                self.is_augmented.append(False)
                
                # Periodic memory check
                if i % 1000 == 0 and i > 0:
                    current_memory = self.check_memory_usage()
                    if current_memory > 11:  # Close to limit
                        print(f"⚠️  Memory usage high: {current_memory:.1f}GB")
                        
            except Exception as e:
                print(f"❌ Error loading {path}: {e}")
                continue
        
        print(f"✅ Loaded {len(self.cached_waveforms)} original samples")
        
        # 2. Create augmented versions for 60% of data
        print("2️⃣ Creating augmented data...")
        num_to_augment = int(len(self.original_paths) * self.aug_ratio)
        
        # Random select samples to augment
        augment_indices = random.sample(range(len(self.original_paths)), num_to_augment)
        
        for i, orig_idx in enumerate(tqdm(augment_indices, desc="Creating augmented")):
            try:
                # Get original waveform
                original_waveform = self.cached_waveforms[orig_idx].copy()
                original_label = self.cached_labels[orig_idx]
                
                # Apply augmentation
                augmented_waveform = self.wav_aug(original_waveform)
                
                # Add to cache
                self.cached_waveforms.append(augmented_waveform.copy())
                self.cached_labels.append(original_label)
                self.is_augmented.append(True)
                
                # Periodic memory check
                if i % 500 == 0 and i > 0:
                    current_memory = self.check_memory_usage()
                    if current_memory > 11:
                        print(f"⚠️  Memory warning: {current_memory:.1f}GB")
                        
            except Exception as e:
                print(f"❌ Error augmenting sample {orig_idx}: {e}")
                continue
        
        # 3. Convert to more memory-efficient format
        print("3️⃣ Optimizing memory usage...")
        
        # Convert lists to numpy arrays (more memory efficient)
        print("   Converting to numpy arrays...")
        final_waveforms = []
        final_labels = []
        final_is_augmented = []
        
        for waveform, label, is_aug in zip(self.cached_waveforms, self.cached_labels, self.is_augmented):
            if waveform is not None:
                final_waveforms.append(waveform.astype(np.float32))  # Use float32 instead of float64
                final_labels.append(label)
                final_is_augmented.append(is_aug)
        
        # Clear original lists to free memory
        del self.cached_waveforms, self.cached_labels, self.is_augmented
        gc.collect()
        
        # Store final arrays
        self.cached_waveforms = final_waveforms
        self.cached_labels = np.array(final_labels)
        self.is_augmented = np.array(final_is_augmented)
        
        final_memory = self.check_memory_usage()
        
        print("\n" + "="*60)
        print("✅ CACHED DATASET CREATION COMPLETED!")
        print("="*60)
        print(f"📊 Total samples: {len(self.cached_waveforms)}")
        print(f"   - Original: {np.sum(~self.is_augmented)}")
        print(f"   - Augmented: {np.sum(self.is_augmented)}")
        print(f"💾 Memory usage: {initial_memory:.1f}GB → {final_memory:.1f}GB")
        print(f"🚀 Ready for fast training!")
        
        return True
        
    def save_cache(self, cache_file="cached_dataset.pkl"):
        """Save cached dataset to disk"""
        cache_path = os.path.join(self.cache_dir, cache_file)
        print(f"💾 Saving cached dataset to {cache_path}...")
        
        cache_data = {
            'waveforms': self.cached_waveforms,
            'labels': self.cached_labels,
            'is_augmented': self.is_augmented,
            'second': self.second,
            'aug_ratio': self.aug_ratio
        }
        
        with open(cache_path, 'wb') as f:
            pickle.dump(cache_data, f, protocol=pickle.HIGHEST_PROTOCOL)
        
        print(f"✅ Cache saved successfully!")
        return cache_path
    
    def load_cache(self, cache_file="cached_dataset.pkl"):
        """Load cached dataset from disk"""
        cache_path = os.path.join(self.cache_dir, cache_file)
        
        if not os.path.exists(cache_path):
            print(f"❌ Cache file not found: {cache_path}")
            return False
        
        print(f"📥 Loading cached dataset from {cache_path}...")
        
        with open(cache_path, 'rb') as f:
            cache_data = pickle.load(f)
        
        self.cached_waveforms = cache_data['waveforms']
        self.cached_labels = cache_data['labels']
        self.is_augmented = cache_data['is_augmented']
        self.second = cache_data['second']
        self.aug_ratio = cache_data['aug_ratio']
        
        print(f"✅ Loaded {len(self.cached_waveforms)} cached samples")
        return True

# Usage
def create_cached_training_data():
    """Main function để tạo cached dataset"""
    
    train_csv_path = "/kaggle/working/train.csv"
    
    # Check if train.csv exists
    if not os.path.exists(train_csv_path):
        print(f"❌ Training CSV not found: {train_csv_path}")
        return False
    
    # Create pre-augmented dataset
    cached_dataset = PreAugmentedDataset(
        train_csv_path=train_csv_path,
        second=3,
        aug_ratio=0.6  # 60% augmentation
    )
    
    # Try to load existing cache first
    cache_file = "vietnam_celeb_cached_60aug.pkl"
    if cached_dataset.load_cache(cache_file):
        print("🎉 Using existing cached dataset!")
    else:
        print("🔧 Creating new cached dataset...")
        success = cached_dataset.create_cached_dataset()
        if success:
            cached_dataset.save_cache(cache_file)
        else:
            print("❌ Failed to create cached dataset")
            return False
    
    return cached_dataset

# Run the caching process
print("🚀 Starting cached dataset creation...")
cached_dataset = create_cached_training_data()

if cached_dataset:
    print(f"\n🎉 SUCCESS! Cached dataset ready with {len(cached_dataset.cached_waveforms)} samples")
    print("Next: Use this for fast training!")
else:
    print("❌ Failed to create cached dataset")

In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np
from sklearn.utils import shuffle

class FastCachedDataset(Dataset):
    """
    Fast dataset class sử dụng pre-cached data
    """
    def __init__(self, cached_dataset, pairs=False, shuffle_data=True):
        self.cached_dataset = cached_dataset
        self.pairs = pairs
        
        # Get cached data
        self.waveforms = cached_dataset.cached_waveforms
        self.labels = cached_dataset.cached_labels
        self.is_augmented = cached_dataset.is_augmented
        
        # Shuffle if requested
        if shuffle_data:
            indices = np.arange(len(self.waveforms))
            np.random.shuffle(indices)
            
            # Apply shuffle
            self.waveforms = [self.waveforms[i] for i in indices]
            self.labels = self.labels[indices]
            self.is_augmented = self.is_augmented[indices]
        
        print(f"🚀 FastCachedDataset initialized:")
        print(f"   📊 Total samples: {len(self.waveforms)}")
        print(f"   📈 Original: {np.sum(~self.is_augmented)}")
        print(f"   🔊 Augmented: {np.sum(self.is_augmented)}")
        print(f"   🔄 Pairs mode: {pairs}")
    
    def __getitem__(self, index):
        # Get cached waveform (already processed)
        waveform_1 = self.waveforms[index]
        label = self.labels[index]
        
        if not self.pairs:
            return torch.FloatTensor(waveform_1), label
        else:
            # For pairs, return same waveform twice (or implement pair logic)
            waveform_2 = waveform_1  # or implement custom pair logic
            return torch.FloatTensor(waveform_1), torch.FloatTensor(waveform_2), label
    
    def __len__(self):
        return len(self.waveforms)
    
    def get_stats(self):
        """Get dataset statistics"""
        stats = {
            'total_samples': len(self.waveforms),
            'original_samples': np.sum(~self.is_augmented),
            'augmented_samples': np.sum(self.is_augmented),
            'unique_speakers': len(np.unique(self.labels)),
            'augmentation_ratio': np.sum(self.is_augmented) / len(self.waveforms)
        }
        return stats

# Test the fast dataset
if 'cached_dataset' in globals() and cached_dataset is not None:
    print("\n🧪 Testing FastCachedDataset...")
    
    fast_dataset = FastCachedDataset(cached_dataset, pairs=False, shuffle_data=True)
    
    # Print stats
    stats = fast_dataset.get_stats()
    print(f"\n📊 Dataset Statistics:")
    for key, value in stats.items():
        if isinstance(value, float):
            print(f"   {key}: {value:.3f}")
        else:
            print(f"   {key}: {value}")
    
    # Test loading speed
    print(f"\n⚡ Speed test...")
    import time
    
    start_time = time.time()
    for i in range(100):  # Test 100 samples
        waveform, label = fast_dataset[i]
    end_time = time.time()
    
    print(f"   ⏱️  100 samples loading time: {(end_time - start_time)*1000:.1f}ms")
    print(f"   🚀 Average per sample: {(end_time - start_time)*10:.1f}ms")
    print(f"   📊 Sample shape: {waveform.shape}")
    
    print("\n✅ FastCachedDataset ready for training!")
else:
    print("❌ Cached dataset not available. Please run the caching process first.")

In [ ]:
import os
import sys
import subprocess
import pandas as pd
import datetime
import torch

# Import fast dataset class
sys.path.append('/kaggle/working/mfa_conformer')

def create_fast_datamodule_patch():
    """
    Patch MFA Conformer để sử dụng FastCachedDataset
    """
    
    # Create new loader.py với FastCachedDataset
    fast_loader_code = '''
import os
import numpy as np
import torch
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader

class FastCachedDataset:
    """Fast dataset from cached data"""
    def __init__(self, cached_dataset, pairs=False):
        self.waveforms = cached_dataset.cached_waveforms
        self.labels = cached_dataset.cached_labels
        self.pairs = pairs
        
        # Shuffle data
        indices = np.arange(len(self.waveforms))
        np.random.shuffle(indices)
        self.waveforms = [self.waveforms[i] for i in indices]
        self.labels = self.labels[indices]
        
        print(f"FastCachedDataset: {len(self.waveforms)} samples loaded")
    
    def __getitem__(self, index):
        waveform_1 = self.waveforms[index]
        label = self.labels[index]
        
        if not self.pairs:
            return torch.FloatTensor(waveform_1), label
        else:
            return torch.FloatTensor(waveform_1), torch.FloatTensor(waveform_1), label
    
    def __len__(self):
        return len(self.waveforms)

class FastSPK_datamodule(LightningDataModule):
    def __init__(self, cached_dataset, trial_path, num_workers=4, batch_size=32, pairs=False, **kwargs):
        super().__init__()
        self.cached_dataset = cached_dataset
        self.trial_path = trial_path
        self.num_workers = num_workers
        self.batch_size = batch_size
        self.pairs = pairs
        
    def train_dataloader(self):
        train_dataset = FastCachedDataset(self.cached_dataset, self.pairs)
        loader = torch.utils.data.DataLoader(
            train_dataset,
            shuffle=True,
            num_workers=self.num_workers,
            batch_size=self.batch_size,
            pin_memory=True,
            drop_last=False,
        )
        return loader
    
    def val_dataloader(self):
        # Import original evaluation dataset
        from .dataset import Evaluation_Dataset
        
        trials = np.loadtxt(self.trial_path, str)
        eval_path = np.unique(np.concatenate((trials.T[1], trials.T[2])))
        print("number of evaluation: {}".format(len(eval_path)))
        
        eval_dataset = Evaluation_Dataset(eval_path, second=-1)
        loader = torch.utils.data.DataLoader(
            eval_dataset,
            num_workers=4,
            shuffle=False, 
            batch_size=1
        )
        return loader
    
    def test_dataloader(self):
        return self.val_dataloader()
'''
    
    # Write to new file
    fast_loader_path = "/kaggle/working/mfa_conformer/module/fast_loader.py"
    with open(fast_loader_path, 'w') as f:
        f.write(fast_loader_code)
    
    print(f"✅ Created fast datamodule at {fast_loader_path}")
    return fast_loader_path

def train_with_cached_data():
    """
    Training với cached data - should be super fast!
    """
    print(f"🚀 Starting FAST training with cached data at: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Check if cached dataset exists
    if 'cached_dataset' not in globals() or cached_dataset is None:
        print("❌ Cached dataset not found! Please run the caching process first.")
        return False
    
    # Create fast datamodule
    fast_loader_path = create_fast_datamodule_patch()
    
    # MFA Conformer path
    mfa_path = "/kaggle/working/mfa_conformer"
    
    # Create modified main.py that uses FastSPK_datamodule
    main_py_path = os.path.join(mfa_path, "main_fast.py")
    
    # Read original main.py
    with open(os.path.join(mfa_path, "main.py"), 'r') as f:
        main_content = f.read()
    
    # Replace datamodule import and usage
    main_content = main_content.replace(
        "from module.loader import SPK_datamodule",
        "from module.fast_loader import FastSPK_datamodule"
    )
    
    # Replace datamodule creation
    old_dm_creation = '''dm = SPK_datamodule(train_csv_path=args.train_csv_path, trial_path=args.trial_path, second=args.second,
            aug=args.aug, batch_size=args.batch_size, num_workers=args.num_workers, pairs=False)'''
    
    new_dm_creation = '''# Use cached dataset instead of CSV
    import pickle
    import sys
    sys.path.append('/kaggle/working')
    
    # Get cached dataset from global scope
    cached_dataset = globals().get('cached_dataset')
    if cached_dataset is None:
        raise ValueError("Cached dataset not found! Please run caching process first.")
    
    dm = FastSPK_datamodule(cached_dataset=cached_dataset, trial_path=args.trial_path, 
                           batch_size=args.batch_size, num_workers=args.num_workers, pairs=False)'''
    
    main_content = main_content.replace(old_dm_creation, new_dm_creation)
    
    # Write modified main.py
    with open(main_py_path, 'w') as f:
        f.write(main_content)
    
    print(f"✅ Created fast training script: {main_py_path}")
    
    # Training parameters
    validation_path = "/kaggle/working/validation.txt"
    save_dir = "/kaggle/working/models"
    
    if not os.path.exists(validation_path):
        print(f"❌ Validation file not found: {validation_path}")
        return False
    
    os.makedirs(save_dir, exist_ok=True)
    
    # Get number of classes
    num_classes = len(np.unique(cached_dataset.cached_labels))
    total_samples = len(cached_dataset.cached_waveforms)
    
    print(f"📊 Training data summary:")
    print(f"   - Total samples: {total_samples}")
    print(f"   - Unique speakers: {num_classes}")
    print(f"   - Augmented ratio: {np.sum(cached_dataset.is_augmented)/total_samples:.1%}")
    
    # Change to MFA directory
    os.chdir(mfa_path)
    
    # Training command - should be much faster now!
    command = [
        "python", "main_fast.py",
        "--accelerator", "gpu",
        "--devices", "1",
        "--batch_size", "200",  # Can use larger batch since no augmentation overhead
        "--num_workers", "6",   # Can use more workers
        "--max_epochs", "25",
        "--embedding_dim", "256",
        "--save_dir", save_dir,
        "--encoder_name", "conformer",
        "--num_blocks", "6",
        "--input_layer", "conv2d",
        "--pos_enc_layer_type", "abs_pos",
        "--loss_name", "amsoftmax",
        "--learning_rate", "0.001",
        "--step_size", "4",
        "--gamma", "0.5",
        "--weight_decay", "0.0000001",
        "--trial_path", validation_path,
        "--num_classes", str(num_classes),
        "--warmup_step", "2000",
        "--precision", "16",
        # NO --aug flag needed since data already augmented!
    ]
    
    print("\n" + "="*60)
    print("⚡ SUPER FAST TRAINING WITH CACHED DATA")
    print("="*60)
    print("Command:", " ".join(command))
    print(f"🔥 Expected time per epoch: ~2-3 minutes (vs 50 minutes before)")
    print(f"💾 All data pre-loaded in RAM - no disk I/O during training!")
    print("="*60)
    
    try:
        print("\n🚀 Starting super fast training...")
        process = subprocess.run(command, check=True)
        print(f"\n🎉 FAST training completed! Return code: {process.returncode}")
        return True
        
    except subprocess.CalledProcessError as e:
        print(f"\n❌ Training failed: {e.returncode}")
        return False

# Run fast training
if __name__ == "__main__":
    success = train_with_cached_data()
    if success:
        print("\n🎉🎉🎉 FAST TRAINING COMPLETED! 🎉🎉🎉")
        print("⚡ Each epoch should now take 2-3 minutes instead of 50!")

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import sys
import time
import glob
import matplotlib.pyplot as plt
from IPython.display import FileLink
from scipy.special import expit
import re

# --------- CONFIGURATIONS ---------
# Paths
USER_CHECKPOINT_PATH = '/kaggle/input/models'  # User can override this
TEST_FILE = '/kaggle/input/voxvietnam/test_list.txt' 
AUDIO_DIR = '/kaggle/input/voxvietnam/wav/wav' 
MODELS_DIR = '/kaggle/working/models'
BEST_OUTPUT_FILE = '/kaggle/working/Best_Predictions.txt'
LAST_OUTPUT_FILE = '/kaggle/working/Last_Predictions.txt'

# Model configurations - must match the trained model
EMBEDDING_DIM = 256
ENCODER_NAME = 'conformer'
NUM_BLOCKS = 6
INPUT_LAYER = 'conv2d'
POS_ENC_LAYER_TYPE = 'abs_pos'
LOSS_NAME = 'amsoftmax'  
NUM_CLASSES = 880  

# Device configuration
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# ---------------------------------------------

print(f"Starting inference at: {time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"User: VietCH57")
print(f"Using device: {DEVICE}")

# Add MFA Conformer path
sys.path.append('/kaggle/working/mfa_conformer')

# Import modules from MFA Conformer
try:
    from main import Task
    from module.dataset import load_audio
    print("Successfully imported modules from mfa_conformer")
except ImportError as e:
    print(f"Error importing modules from mfa_conformer: {e}")
    print("Please check the mfa_conformer directory")
    raise

# Create a dummy trial file
dummy_trial_path = '/kaggle/working/dummy_trial.txt'
with open(dummy_trial_path, 'w') as f:
    f.write("1 /path/to/dummy1 /path/to/dummy2\n")
# ---------------------------------------------

def extract_embeddings(model, audio_paths, device='cuda'):
    """Extract embeddings from audio files"""
    embeddings = {}
    model.to(device)
    model.eval()
    
    error_count = 0
    
    for path in tqdm(audio_paths, desc="Extracting embeddings"):
        try:
            # Load audio with proper error checking
            waveform = load_audio(path, second=-1)  # Use full audio (-1)
            
            # Basic checks on the waveform
            if waveform is None or len(waveform) == 0:
                print(f"Warning: Empty audio for {path}")
                embeddings[path] = np.zeros(model.hparams.embedding_dim)
                error_count += 1
                continue
                
            # Convert to tensor and move to device
            waveform = torch.FloatTensor(waveform).unsqueeze(0).to(device)
            
            # Extract embedding
            with torch.no_grad():
                outputs = model(waveform)
                
                # Handle different output types
                if isinstance(outputs, tuple):
                    embedding = outputs[0]  # Assume first element is embedding
                else:
                    embedding = outputs
                    
                # Convert to numpy and flatten if needed
                embedding = embedding.cpu().numpy()
                if embedding.ndim > 1:
                    embedding = embedding[0]  # Get the first embedding if batch
            
            # Store embedding with audio path as key
            embeddings[path] = embedding
            
        except Exception as e:
            print(f"Error processing {path}: {str(e)}")
            embeddings[path] = np.zeros(model.hparams.embedding_dim)
            error_count += 1
    
    # Report error rate
    if error_count > 0:
        print(f"Warning: {error_count}/{len(audio_paths)} files ({error_count/len(audio_paths)*100:.2f}%) failed to process properly")
    
    return embeddings

def compute_score(emb1, emb2):
    """Compute cosine similarity between two embeddings and normalize to [0, 1]"""
    # Normalize embeddings
    emb1 = emb1 - np.mean(emb1)
    emb2 = emb2 - np.mean(emb2)
    
    # Compute cosine similarity
    score = np.dot(emb1, emb2)
    denom = np.linalg.norm(emb1) * np.linalg.norm(emb2)
    if denom > 0:
        score = score / denom
    else:
        score = 0.0
    
    # Normalize to [0, 1]
    score = expit(score)  # Use sigmoid to normalize

    return score

def find_best_checkpoint(models_dir=MODELS_DIR):
    """Find the checkpoint with lowest EER value"""
    best_ckpt = None
    best_eer = float('inf')
    
    # Find all checkpoint files
    all_ckpts = glob.glob(os.path.join(models_dir, '*.ckpt'))
    
    if not all_ckpts:
        return None
    
    # Look for checkpoints with "eer" in filename
    for ckpt in all_ckpts:
        # Try to extract EER value using regex to handle format like "epoch=7_cosine_eer=9.48.ckpt"
        match = re.search(r'eer=?(\d+\.\d+)', os.path.basename(ckpt))
        if match:
            try:
                eer = float(match.group(1))
                if eer < best_eer:
                    best_eer = eer
                    best_ckpt = ckpt
            except ValueError:
                continue
    
    if best_ckpt:
        print(f"Found best checkpoint with EER = {best_eer}: {os.path.basename(best_ckpt)}")
    else:
        print(f"No checkpoint with EER found in {models_dir}")
    
    return best_ckpt

def find_last_checkpoint(models_dir=MODELS_DIR):
    """Find the checkpoint with highest epoch number"""
    last_ckpt = None
    highest_epoch = -1
    
    # Find all checkpoint files
    all_ckpts = glob.glob(os.path.join(models_dir, '*.ckpt'))
    
    if not all_ckpts:
        return None
    
    # Look for epoch number in filenames
    for ckpt in all_ckpts:
        # Try to extract epoch number using regex
        match = re.search(r'epoch=?(\d+)', os.path.basename(ckpt))
        if match:
            try:
                epoch = int(match.group(1))
                if epoch > highest_epoch:
                    highest_epoch = epoch
                    last_ckpt = ckpt
            except ValueError:
                continue
    
    # If no epoch found in filenames, use last modified file
    if last_ckpt is None:
        # Sort by modification time (newest first)
        last_ckpt = sorted(all_ckpts, key=os.path.getmtime, reverse=True)[0]
        print(f"No epoch number found in filenames, using most recently modified: {os.path.basename(last_ckpt)}")
    else:
        print(f"Found last checkpoint with epoch = {highest_epoch}: {os.path.basename(last_ckpt)}")
    
    return last_ckpt

def run_inference(checkpoint_path, output_file):
    """Run inference using the specified checkpoint and save to output file"""
    print(f"\n{'='*50}")
    print(f"Running inference with checkpoint: {os.path.basename(checkpoint_path)}")
    print(f"{'='*50}")
    
    # Initialize model with all required parameters
    print(f"Loading model from {checkpoint_path}...")
    model = Task(
        embedding_dim=EMBEDDING_DIM,
        encoder_name=ENCODER_NAME,
        num_blocks=NUM_BLOCKS,
        input_layer=INPUT_LAYER,
        pos_enc_layer_type=POS_ENC_LAYER_TYPE,
        trial_path=dummy_trial_path,
        loss_name=LOSS_NAME,  
        num_classes=NUM_CLASSES,  
        learning_rate=0.001,  
        weight_decay=0.0000001,  
        batch_size=100,  
        num_workers=2,  
        max_epochs=50,  
    )

    # Load trained weights
    try:
        checkpoint = torch.load(checkpoint_path, map_location="cpu")
        model.load_state_dict(checkpoint["state_dict"], strict=False)
        print("Model loaded successfully!")
    except Exception as e:
        print(f"Error loading model: {e}")
        return False

    # Extract embeddings for all audio files
    print(f"Extracting embeddings using {DEVICE}...")
    embeddings = extract_embeddings(model, all_audio_files, device=DEVICE)
    print(f"Extracted embeddings for {len(embeddings)} audio files")

    # Compute similarity scores for each pair
    print("Computing similarity scores...")
    scores = []
    for audio1, audio2 in tqdm(audio_pairs, desc="Computing scores"):
        emb1 = embeddings[audio1]
        emb2 = embeddings[audio2]
        score = compute_score(emb1, emb2)
        scores.append(score)

    # Save scores to output file
    os.makedirs(os.path.dirname(os.path.abspath(output_file)), exist_ok=True)
    with open(output_file, 'w') as f:
        for score in scores:
            f.write(f"{score:.6f}\n")

    # Analyze score distribution
    scores_np = np.array(scores)
    print(f"Scores saved to {output_file}")
    print(f"Number of scores: {len(scores)}")
    print(f"Score range: {np.min(scores):.6f} to {np.max(scores):.6f}")
    print(f"Score mean: {np.mean(scores):.6f}")
    print(f"Score standard deviation: {np.std(scores):.6f}")

    # Plot score distribution
    plt.figure(figsize=(12, 6))
    plt.hist(scores_np, bins=50)
    plt.title(f'Distribution of Similarity Scores\n{os.path.basename(checkpoint_path)}')
    plt.xlabel('Score')
    plt.ylabel('Count')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    plot_filename = os.path.basename(output_file).split('.')[0] + '_distribution.png'
    output_plot = os.path.join('/kaggle/working', plot_filename)
    plt.savefig(output_plot)
    print(f"Score distribution plot saved to {output_plot}")
    plt.close()  # Close the plot to avoid displaying multiple plots

    # Display sample scores
    print("\nSample of scores (first 5):")
    for i, score in enumerate(scores[:5]):
        print(f"  {i+1}: {score:.6f}")
        
    return True

# Check if test file exists
if not os.path.exists(TEST_FILE):
    print(f"Test file not found at {TEST_FILE}")
    raise FileNotFoundError(f"Test file not found at {TEST_FILE}")

# Read test file
audio_pairs = []
with open(TEST_FILE, 'r') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) != 2:
            print(f"Warning: Invalid line in test file: {line.strip()}")
            continue
            
        audio1 = os.path.join(AUDIO_DIR, parts[0])
        audio2 = os.path.join(AUDIO_DIR, parts[1])
        audio_pairs.append((audio1, audio2))

print(f"Found {len(audio_pairs)} audio pairs in test file")

# Get all unique audio files
all_audio_files = []
for pair in audio_pairs:
    all_audio_files.extend(pair)
all_audio_files = list(set(all_audio_files))
print(f"Found {len(all_audio_files)} unique audio files")

# Check if audio files exist
missing_files = [path for path in all_audio_files if not os.path.exists(path)]
if missing_files:
    print(f"Warning: {len(missing_files)} audio files not found")
    if len(missing_files) < 10:
        for file in missing_files:
            print(f"  Missing: {file}")
    else:
        for file in missing_files[:5]:
            print(f"  Missing: {file}")
        print(f"  ... and {len(missing_files) - 5} more")

# Find checkpoints
user_ckpt = None
if os.path.exists(USER_CHECKPOINT_PATH):
    # If user provided a directory, find best checkpoint there
    if os.path.isdir(USER_CHECKPOINT_PATH):
        user_ckpt = find_best_checkpoint(USER_CHECKPOINT_PATH)
    # If user provided a specific checkpoint file
    elif USER_CHECKPOINT_PATH.endswith('.ckpt') and os.path.isfile(USER_CHECKPOINT_PATH):
        user_ckpt = USER_CHECKPOINT_PATH
    else:
        print(f"User checkpoint not found at {USER_CHECKPOINT_PATH}")

# Find best and last checkpoints in working models directory
best_ckpt = find_best_checkpoint()
last_ckpt = find_last_checkpoint()

# Track which checkpoints we've already processed
processed_checkpoints = set()

# Process user checkpoint if provided
if user_ckpt:
    print(f"\nUsing user-specified checkpoint: {os.path.basename(user_ckpt)}")
    user_output_file = '/kaggle/working/User_Predictions.txt'
    run_inference(user_ckpt, user_output_file)
    processed_checkpoints.add(user_ckpt)

# Process best checkpoint
if best_ckpt and best_ckpt not in processed_checkpoints:
    run_inference(best_ckpt, BEST_OUTPUT_FILE)
    processed_checkpoints.add(best_ckpt)
else:
    if best_ckpt in processed_checkpoints:
        print(f"\nBest checkpoint already processed: {os.path.basename(best_ckpt)}")
    else:
        print("\nNo best checkpoint found.")

# Process last checkpoint
if last_ckpt and last_ckpt not in processed_checkpoints:
    run_inference(last_ckpt, LAST_OUTPUT_FILE)
    processed_checkpoints.add(last_ckpt)
else:
    if last_ckpt in processed_checkpoints:
        print(f"\nLast checkpoint already processed: {os.path.basename(last_ckpt)}")
    else:
        print("\nNo last checkpoint found.")

print(f"\nInference completed at: {time.strftime('%Y-%m-%d %H:%M:%S')}")

# Create download links for prediction files
print("\nPrediction files ready for download:")
for output_file in [BEST_OUTPUT_FILE, LAST_OUTPUT_FILE, '/kaggle/working/User_Predictions.txt']:
    if os.path.exists(output_file):
        display(FileLink(output_file))